# Movie_Classification.py

### Tensorflow, pandas 라이브러리 추가

In [128]:
#Tensorflow 라이브러리 추가
import tensorflow as tf

#pandas 라이브러리 추가
import pandas as pd

#sklearn의 train_test_split 라이브러리 추가(Dataset 내에서 train data와 test data를 분리하기 위해 사용)
from sklearn.model_selection import train_test_split

### Data Set 불러오기

In [129]:
#data-set read
movie = pd.read_csv("./data/movie.csv", encoding ='CP949')

#data set을 동아시아 글자 간격으로 보이게 설정
pd.set_option('display.unicode.east_asian_width', True)

### Data set의 열을 분리(Slice)

In [130]:
#label 데이터를 제외한 영화관련 데이터
movie_data = movie.iloc[:,:-1]

print(movie_data.head())

#맨 마지막 열인 label 데이터
label_data = movie.iloc[:,-1:]

print(label_data.head())

             영화제목  직감  이야기  감독경력  동원경험  배우  관람객 평점  \
0    블레이드러너2049     9      11        14         1    14            8   
1          쥬라기월드    16      15        11         1    14            9   
2                독전    15      12         6         1    13            8   
3             데드풀2    16      16        10         1    14            9   
4  어벤저스인피니티워    19      19        10         1    15            9   

   전체 댓글 수  관객 수  
0             5       32  
1             5      566  
2             5      370  
3             5      378  
4             5     1121  
   Classes
0        0
1        1
2        1
3        1
4        1


### 전체 data set중 30%를 test데이터, 70%는 학습에 사용하기 위해 임의로 분할

In [131]:
train_x, test_x, train_y, test_y = train_test_split(movie_data, label_data, test_size=0.3, random_state=42, stratify=label_data)

### 첫번째 열인 영화제목 열을 분리

In [132]:
train_name = train_x['영화제목']
test_name = test_x['영화제목']

print(train_name.head())

print(test_name.head())

16                  앤트맨과 와스프
21                     캡틴아메리카
11    극장판 헬로카봇 : 백악기 시대
22                             밀정
8                          램페이지
Name: 영화제목, dtype: object
14    어느 가족
40       챔피온
3       데드풀2
15         인랑
29         1987
Name: 영화제목, dtype: object


### 직감 ~ 전체 댓글 수 열을 학습에 사용하기 위해 현재 가장 마지막인 관객 수 열을 분리 

In [133]:
train_x = train_x.iloc[:,1:-1]

print(train_x.head())
print("\n")
test_x = test_x.iloc[:,1:-1]

print(test_x.head())

    직감  이야기  감독경력  동원경험  배우  관람객 평점  전체 댓글 수
16    11      12        14         1    11            9             5
21     8       6         4         1    12            9             5
11    10      10        10         1     9            9             4
22    13      14        22         1    15            9             3
8     12      12         9         1    12            9             5


    직감  이야기  감독경력  동원경험  배우  관람객 평점  전체 댓글 수
14    10      14        13         1     9            9             4
40    13       9         5         0    12            8             5
3     16      16        10         1    14            9             5
15     9       8        15         1    15            6             5
29    16      16         7         1    14            9             5


### tensorflow 라이브러리를 이용하여 네트워크 만들기
#### Input Layer 노드의 수 : 7개
#### 1번째 Hidden Layer 노드의 수 : 3개
#### 2번째 Hidden Layer 노드의 수 : 2개
#### Output Layer 노드의 수 : 1개

In [134]:
#Tensorflow에서 값을 처리하기 위해 필요한 Session 유형의 변수를 sess로 생성
sess = tf.Session()

#9개의 픽셀 데이터에 대한 Tensor
#shape=(None, 7) : 입력하는 개수가 많을때는 정확히 알 수 없음(None), 한 번에 입력할 때 들어가는 데이터의 특징의 수 : 픽셀 9칸 -> 9
#Input Layer 노드의 수 = 7
X = tf.placeholder(tf.float32, shape=([None,7]),name="X")

#X에 입력한 픽셀의 실제 Label 값에 대한 Tensor
#shape=(None, 1) : Output Layer의 노드의 수 = 1
Y = tf.placeholder(tf.int32, shape=([None,1]),name="Y")

#Output Layer 노드의 수가 1개로 0인지 1인지 구별하기 위해 사용하는 방법이 One-hot encoding
# 250만이상 관객 수 동원에 대한 One-hot encoding
Y_one_hot = tf.one_hot(Y, 2, name="Y_one_hot")  

#[-1, 2]의 형태로 변환 -> -1 : 몇 개의 데이터를 사용할 지 모른다는 의미, 2 : 0, 1로 구별하겠다는 의미 
Y_one_hot = tf.reshape(Y_one_hot, [-1, 2])

#Hidden1_Layer
#Input Layer의 각 노드에 대한 가중치(Weight)
#W1 : 첫번째 Hidden Layer의 노드 3개
W1 = tf.Variable(tf.truncated_normal([7,3]),name='W1')
        
#Input Layer의 각 노드에 대한 편향(bias)
#b1 : 첫번째 Hidden Layer의 각 노드의 bias
b1 = tf.Variable(tf.truncated_normal([3]),name='b1')

#Hidden1_Layer : 입력한 데이터와 가중치의 곱셈 결과 + 편향(bias)
H1_logits = tf.matmul(X, W1) + b1

#Hidden2_Layer :  Hidden1_Layer결과값과 가중치의 곱셈 결과 + 편향(bias)
W2 = tf.Variable(tf.truncated_normal([3,2]),name="W2")

b2 = tf.Variable(tf.truncated_normal([2]), name='b2')

#Hidden2_Layer 계산 ㄱ밧
logits = tf.matmul(H1_logits, W2) + b2

#입력데이터와 출력 데이터의 관계(Relationship) 또는 패턴(Pattern)을 나타내기 위한 함수 : hypothesis
hypothesis = tf.nn.softmax(logits)

#Logits를 통해 그려진 그래프와, hypothesis를 통해 판별한 결과의 오차를 계산
cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y_one_hot)

#전체 오차의 평균
cost = tf.reduce_mean(cost_i) 

#경사하강법(Gradient-Descent)를 이용하여 학습
#학습률(Learning_rate) : 0.05 -> 학습을 하는 과정에서 경사를 하강하는 폭의 정도 -> 작을 수록 폭이 좁음, 넓을 수록 폭이 넓음
optimization = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(cost)

#우리가 선정한 hypothesis 함수를 기반으로 Classification한 결과
prediction = tf.argmax(hypothesis, 1) 

#Prediction의 결과가 실제 Label 값과 맞는지 여부
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1)) 

#Prediction의 정확성을 저장하는 변수
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) 

#Tensorflow의 변수 초기화
sess.run(tf.global_variables_initializer()) 

### 구성한 네트워크를 기반으로 2001번 반복하며 학습하기

In [135]:
#2001번 반복하여 학습
for step in range(2001):
    
    #입력은 train_x, Prediction 결과 비교를 위한 Y는 train_y로 사용하여 학습
    _ = sess.run(optimization, feed_dict={X:train_x,Y:train_y}) 
    
    
    if step % 1000 == 0:
        loss, acc = sess.run([cost, accuracy], feed_dict={X: train_x, Y: train_y})
        print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))

Step:     0	Loss: 8.051	Acc: 36.67%
Step:  1000	Loss: 0.548	Acc: 73.33%
Step:  2000	Loss: 0.543	Acc: 76.67%


### 학습한 결과 테스트하기

In [140]:
test_acc,test_predict,test_correct = sess.run([accuracy,prediction,correct_prediction], feed_dict={X: test_x, Y: test_y})

print("Test Prediction =", test_acc)   

#Test 결과를 저장하기 위한 Data구조 생성
sub = pd.DataFrame() 

#영화제목
sub['Movie Title'] = test_name

#모델이 예측한 값 :250만 넘으면 1, 그렇지 않으면 0
sub['Predict_Audience'] = test_predict 

#실제 관객 동원 수 : 250만 넘으면 1, 그렇지 않으면 0
sub['Origin_Audience'] = test_y

#모델이 예측한 값과 실제 Label 값이 맞는지 여부
sub['Correct'] = test_correct

print(sub)

Test Prediction = 0.61538464
          Movie Title  Predict_Audience  Origin_Audience  Correct
14          어느 가족                 0                0     True
40             챔피온                 1                0    False
3             데드풀2                 1                1     True
15               인랑                 1                0    False
29               1987                 1                1     True
2                독전                 1                1     True
20           검사외전                 0                1    False
10  신과함께 - 인과연                 1                1     True
7    레디 플레이어 원                 1                0    False
34           그래비티                 1                1     True
13       인크레더블 2                 1                1     True
32               더킹                 1                1     True
31           7년의 밤                 1                0    False
